In [28]:
import csv
import numpy as np
import math

In [29]:
iris_file_path = "data/iris.data"

iris_classes = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]

In [30]:
def read_data(file_path, classes):
    matrix = list()
    
    with open(file_path, newline='') as file:
        reader = csv.reader(file, delimiter=',', quotechar='|')
        
        for row in reader:
            matrix_row = [1]
            matrix_row += [float(x) for x in row[:-1]] 
            for class_index in range(len(classes)):
                if classes[class_index] == row[4]:
                    matrix_row += [class_index + 1]
                    
            matrix.append(matrix_row)
                
    return matrix

In [31]:
def classify(x, w):
    return 1 if sigma(x.dot(w)) > 0.5 else 0
    

def calculate_error(x, y, w, tau):
    count = x.shape[0]
    incorrect_count = 0
    
    for j in range(count):
        point = classify(x[j], w)
        if point != y[j][0]:
            incorrect_count += 1
            
    return incorrect_count / count
        
        
def calcualte_cost(w, x, y, tau):
    assert x.shape[0] == y.shape[0]
    
    log_func_v = sigma(x.dot(w))
    y = np.squeeze(y)
    step1 = y * np.log(log_func_v)
    step2 = (1 - y)*np.log(1 - log_func_v)
    final = -step1 - step2 
    return np.mean(final) + tau*w.dot(w)

In [32]:
def sigma(z):
    return 1 / (1 + math.e**(-z))


def sgd_learn(etta, T, tau, trainingX, trainingY):
    w = np.array([0, 0, 0, 0, 0])
    weight_sum = w
    for i in range(T):
        j = np.random.randint(0, trainingX.shape[0])
        x = trainingX[j]
        y = trainingY[j]
        a = sigma(x.dot(w))
        w = w - etta*(a - y)*x - etta*2*tau*w
        weight_sum = weight_sum + w
    return weight_sum / T


def bgd_learn(etta, T, tau, x, y):
    w = np.matrix([0, 0, 0, 0, 0])
    weight_sum = w
    for i in range(T):
        z = np.squeeze(np.asarray(np.dot(x, w.T).T))
        a = np.array(sigma(z))  
        dz = a - y.T
        dw = np.dot(dz, x)
        w = w - etta*dw/x.shape[0] - etta*2*tau*w
        weight_sum = weight_sum + w
    return np.squeeze(np.asarray(w))
    
    
def k_fold(folds_count, x, y, tau, etta, T, algorithm):
    mk = x.shape[0] // folds_count
    
    error = 0
    for i in range(folds_count):
        validationX = x[i * mk:(i + 1)*mk, :]
        validationY = y[i * mk:(i + 1)*mk, :]
        
        trainX = np.concatenate((x[:i * mk, :], x[(i + 1)*mk:, :]),axis = 0)
        trainY = np.concatenate((y[:i * mk, :], y[(i + 1)*mk:, :]),axis = 0)
        
        w = algorithm(etta, T, tau, trainX, trainY)
        
        error += calcualte_cost(w, x, y, tau)
        
    return error / folds_count

In [33]:
def k_fold_learn(etta_values, T_values, tau_values, x, algorithm, folds_count, iris_num):
    trainingX = x[:9 * x.shape[0] // 10, :5]
    trainingY = x[:9 * x.shape[0] // 10, 5:]
    
    np.place(trainingY, trainingY != iris_num, [0])
    np.place(trainingY, trainingY == iris_num, [1])
    
    minL = 1000000
    bestEtta = 0
    bestT = 0
    bestTau = 0
    for etta in etta_values:
        for tau in tau_values:
            for T in T_values:
                
                L = k_fold(folds_count, trainingX, trainingY, tau, etta, T, algorithm)
                
                if(L < minL):
                    minL = L
                    bestT = T
                    bestEtta = etta
                    bestTau = tau
    return bestEtta, bestT, bestTau

        
def model(etta, T, tau, x, iris_num, algorithm):
    trainingX = x[:9 * x.shape[0] // 10, :5]
    validationX = x[9 * x.shape[0] // 10:, :5]
    trainingY = x[:9 * x.shape[0] // 10, 5:]
    validationY = x[9 * x.shape[0] // 10:, 5:]
    
    np.place(trainingY, trainingY != iris_num, [0])
    np.place(trainingY, trainingY == iris_num, [1])
    np.place(validationY, validationY != iris_num, [0])
    np.place(validationY, validationY == iris_num, [1])
    
    w = algorithm(etta, T, tau, trainingX, trainingY)
    
    error = calculate_error(validationX, validationY, w, tau)
    print ("Classification error = ", error)
    
    return w


data = read_data(iris_file_path, iris_classes)
x = np.array(data)
np.random.shuffle(x)

etta_values = [0.001, 0.005, 0.01, 0.1, 0.3, 0.5]
tau_values = [0.001, 0.005, 0.01, 0.1, 0.3, 0.5]
T_values = [20, 50, 100, 200, 300]
for i in range(1, 4):
    print (iris_classes[i - 1])
    (sgdEtta, sgdT, sgdTau) = k_fold_learn(etta_values, T_values, tau_values, x.copy(), sgd_learn, 9, i)
    w = model(sgdEtta, sgdT, sgdTau, x.copy(), i, sgd_learn)
    print("Best params for SGD = ", (sgdTau, sgdEtta, sgdT))
    print("Best weights = ", w)
    (bgdEtta, bgdT, bgdTau) = k_fold_learn(etta_values, T_values, tau_values, x.copy(), bgd_learn, 9, i)
    w = model(bgdEtta, bgdT, bgdTau, x.copy(), i, bgd_learn)
    print("Best params for BGD = ", (bgdTau, bgdEtta, bgdT))
    print("Best weights = ", w)

Iris-setosa
Classification error =  0.0
Best params for SGD =  (0.001, 0.3, 300)
Best weights =  [ 0.2773781   0.42512553  1.40396133 -2.34996569 -1.17619898]
Classification error =  0.0
Best params for BGD =  (0.001, 0.5, 300)
Best weights =  [ 0.31826334  0.53358791  1.74690781 -2.75848338 -1.24942991]
Iris-versicolor
Classification error =  0.26666666666666666
Best params for SGD =  (0.001, 0.01, 300)
Best weights =  [-0.02571743 -0.10190194 -0.19999495  0.12243666  0.0211853 ]
Classification error =  0.26666666666666666
Best params for BGD =  (0.001, 0.1, 300)
Best weights =  [ 0.21748424  0.19472883 -1.00404741  0.42285113 -0.57424994]
Iris-virginica
Classification error =  0.0
Best params for SGD =  (0.001, 0.1, 300)
Best weights =  [-0.43893728 -1.18681137 -0.98582995  1.82856872  1.37313154]
Classification error =  0.0
Best params for BGD =  (0.001, 0.5, 300)
Best weights =  [-1.57041013 -3.00103456 -2.68388365  4.3496805   3.96329202]
